<a href="https://colab.research.google.com/github/luigimersico/Topic_modelling/blob/main/detect_potential_structures_of_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pdfplumber spacy scikit-learn matplotlib pyPDF2 xlsxwriter PyMuPDF keyboard

In [135]:
import os
import pdfplumber
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
from PyPDF2 import PdfReader
import csv
import re
import xlsxwriter
import fitz
import pathlib
import keyboard
import time
from pathlib import Path
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
import numpy as np

 activites: 1) read multiple .pdf file of scientific article. Articles have authors, title, year of publications and corpus. 2) create a variable storing the name of the file. 3) extract from the pdf files title, authors, year of publications and corpus 3) dataframe file with 4 columns: 1 ID of article 2 year of publications 3) authors 4) title 5) corpus. Add the extracted text in the right column of csv file

In [77]:
# Specify the directory where your PDF files are located
pdf_folder_path = '/content/drive/MyDrive/Colab_Notebooks/input_data/example'
# Specify the directory to save the converted text files
text_output_folder = '/content/drive/MyDrive/Colab_Notebooks/output_data'

In [78]:
def get_files(folder):
    files = [f for f in os.listdir(folder) if f.endswith(".pdf")]
    return files

In [80]:
# Get a list of all PDF files in the specified directory
pdf_file_names = get_files(pdf_folder_path)

In [81]:
def convert_pdf_to_text(pdf_path, output_folder):
    pdf_reader = PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    txt_file_path = os.path.join(output_folder, os.path.splitext(os.path.basename(pdf_path))[0] + ".txt")
    with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

In [85]:
# Convert each PDF file to text and save in the text output folder
for pdf_file_name in pdf_file_names:
    pdf_file_path = os.path.join(pdf_folder_path, pdf_file_name)
    convert_pdf_to_text(pdf_file_path, text_output_folder)

In [ ]:
# Load spaCy's pre-trained model
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

In [98]:
# Function to extract text from a text file
def extract_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

In [143]:
# Function to detect structure using KMeans clustering
def detect_structure(text):
    doc = nlp(text)

    # Extract sentences or tokens for clustering
    sentences = [sent.text for sent in doc.sents]

    # Convert text to features using CountVectorizer
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(sentences)

    # Optimize the number of clusters using the elbow method
    scores = []
    max_clusters = 100  # modify on the basis of your needs
    for n_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(X)
        score = silhouette_score(X, kmeans.labels_)
        scores.append(score)

    optimal_clusters = np.argmax(scores) + 2  # Adding 2 because we started from 2 clusters

    # Choose the optimal clustering technique based on silhouette score
    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    agglomerative = AgglomerativeClustering(n_clusters=optimal_clusters)

    kmeans.fit(X)
    agglomerative.fit(X.toarray())

    if silhouette_score(X, kmeans.labels_) > silhouette_score(X, agglomerative.labels_):
        labels = kmeans.labels_
    else:
        labels = agglomerative.labels_

    # Assign clusters to sentences
    for i, label in enumerate(labels):
        sentences[i] = (label, sentences[i])

    return sentences

# Function to process and detect structure in multiple text files
def process_and_detect_structure(txt_folder):
    detected_structures = {}

    for filename in os.listdir(txt_folder):
        if filename.endswith('.txt'):
            txt_path = os.path.join(txt_folder, filename)
            txt_text = extract_text_from_file(txt_path)
            potential_structure = detect_structure(txt_text)
            detected_structures[filename] = potential_structure

    return detected_structures

In [100]:
text_folder_path = '/content/drive/MyDrive/Colab_Notebooks/output_data'

In [ ]:
# Process PDFs and detect potential structures
detected_structures = process_and_detect_structure(text_folder_path)


In [ ]:
# Print the detected structures for each file
for filename, structures in detected_structures.items():
    print(f"Detected Structures for {filename}:")
    for label, sentence in structures:
        print(f"Cluster {label}: {sentence}")
    print("\n")

In [ ]:
for filename, structure in detected_structures.items():
    # Initialize variables to store extracted information
    journal = ""
    authors = ""
    title = ""
    year = ""
    abstract = ""
    corpus = ""

    # Iterate through the structure to extract information
    for label, sentence in structure:
        if "journal" in sentence.lower():
            journal = sentence
        elif "authors" in sentence.lower():
            authors = sentence.replace("Authors:", "").strip()
        elif "title" in sentence.lower():
            title = sentence.replace("Title:", "").strip()
        elif re.search(r"\b(?:19|20)\d{2}\b", sentence):
            year = re.search(r"\b(?:19|20)\d{2}\b", sentence).group()
        elif "abstract" in sentence.lower():
            abstract = sentence.replace("ABSTRACT", "").strip()
        elif "introduction" in sentence.lower():
            corpus_start = structure.index((label, sentence)) + 1
            break

    # Extract corpus by concatenating sentences from corpus_start
    corpus_sentences = [sentence for _, sentence in structure[corpus_start:]]
    corpus = " ".join(corpus_sentences)

    # Print or store the extracted information
    print("Filename:", filename)
    print("Journal:", journal)
    print("Authors:", authors)
    print("Title:", title)
    print("Year:", year)
    print("Abstract:", abstract)
    print("Corpus:", corpus)
    print("\n")

In [ ]:
 # Print or store the extracted information
    print("Filename:", filename)
    print("Journal:", journal)
    print("Authors:", authors)
    print("Title:", title)
    print("Year:", year)
    print("Abstract:", abstract)
    print("Corpus:", corpus)
    print("\n")